In [ ]:
import osiris_utils as ou
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
plt.rcParams['font.size'] = 14

# mpl.rcParams.update({
#     "figure.figsize": (6.0, 4.0),   # same width×height (inches) for all figures
#     "figure.dpi": 200,              # on-screen
#     "savefig.dpi": 300,             # export DPI
# })

from scipy.stats import linregress
from scipy.interpolate import RegularGridInterpolator
from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator,  FormatStrFormatter
from scipy import optimize
from mpl_toolkits.mplot3d import Axes3D

import contextlib
import io
import math


In [ ]:
def UtoV(u):
    return u / np.sqrt(1 + u**2)

def VtoU(v):
    return v * np.sqrt(1 - v**2)

def UtoGamma(u):
    return np.sqrt(1 + u**2)

In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_p(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["p1"][particle, :][mask],
        track.data["p2"][particle, :][mask],
        track.data["p3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj


def get_fields(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["E1"][particle, :][mask],
        track.data["E2"][particle, :][mask],
        track.data["E3"][particle, :][mask],
        track.data["B1"][particle, :][mask],
        track.data["B2"][particle, :][mask],
        track.data["B3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_gradB(sim, t=None):
    
    # Grid centered simulation
    sim_centered = ou.FieldCentering_Simulation(sim)

    B = (sim_centered["part_b1"]**2 + sim_centered["part_b2"]**2 + sim_centered["part_b3"]**2)**(0.5)

    dB_x1 = ou.Derivative_Diagnostic(B, "x1")
    dB_x2 = ou.Derivative_Diagnostic(B, "x2")
    dB_x3 = ou.Derivative_Diagnostic(B, "x3")
    
    if isinstance(t, int):
        return np.stack([
            dB_x1[t],
            dB_x2[t],
            dB_x3[t]
        ], axis=0)
    else:
        return np.stack([
            dB_x1,
            dB_x2,
            dB_x3
        ], axis=0)

def get_b(sim, t=None):
    # Grid centered simulation
    sim_centered = ou.FieldCentering_Simulation(sim)

    B = (sim_centered["part_b1"]**2 + sim_centered["part_b2"]**2 + sim_centered["part_b3"]**2)**(0.5)

    b1 = sim_centered["part_b1"] / B
    b2 = sim_centered["part_b2"] / B
    b3 = sim_centered["part_b3"] / B

    b = np.stack([b1, b2, b3], axis=0)

    if isinstance(t, int):
        return b[t]
    else:
        return b
    
def get_Jacobian_b(sim, t=None):
    
    # Grid centered simulation
    sim_centered = ou.FieldCentering_Simulation(sim)

    B = (sim_centered["part_b1"]**2 + sim_centered["part_b2"]**2 + sim_centered["part_b3"]**2)**(0.5)

    b1 = sim_centered["part_b1"] / B
    b2 = sim_centered["part_b2"] / B
    b3 = sim_centered["part_b3"] / B

    J_B = np.empty((3, 3), dtype=object)
    for i, b_buffer in enumerate([b1, b2, b3]):
        for j, axis in enumerate(["x1", "x2", "x3"]):
            J_B[i,j] = ou.Derivative_Diagnostic(b_buffer, axis)

    if isinstance(t, int):
        return J_B[:, :, t]
    else:
        return J_B


def get_Jacobian_b_Interp(J_B, t=None):
    vfunc = np.vectorize(diag_interpolator, otypes=[object])
    return vfunc(J_B, t=t)


def diag_interpolator(diag, t=0):
    # TODO maybe considder periodic boundaries
    
    # Define grid points
    x = np.linspace(diag.grid[0][0] + diag.dx[0] / 2., diag.grid[0][1] - diag.dx[0] / 2., diag.nx[0])
    y = np.linspace(diag.grid[1][0] + diag.dx[1] / 2., diag.grid[1][1] - diag.dx[1] / 2., diag.nx[1])
    z = np.linspace(diag.grid[2][0] + diag.dx[2] / 2., diag.grid[2][1] - diag.dx[2] / 2., diag.nx[2])

    # return RegularGridInterpolator((x, y, z), diag[t], bounds_error=False, fill_value=None, method='cubic')
    return RegularGridInterpolator((x, y, z), diag[t], bounds_error=False, fill_value=None, method='linear')

In [ ]:
def createSimDic(sim_labels, test):
    sim = {}
    for key in sim_labels.keys():
        sim[key] = {}
        for dtw in sim_labels[key]:
            sim[key][dtw] = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/{key}/dtw{dtw}/{key}.in")
    return sim

def p_parallel(sim, pusher, dtw, particle):
    B1  = sim[pusher][dtw]["test_electrons"]["tracks"]["B1"][particle,:]
    B2  = sim[pusher][dtw]["test_electrons"]["tracks"]["B2"][particle,:]
    B3  = sim[pusher][dtw]["test_electrons"]["tracks"]["B3"][particle,:]
    Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

    p1 = sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,:]
    p2 = sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,:]
    p3 = sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,:]

    p_par = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
    return p_par


def x_parallel(sim, pusher, dtw, particle):
    B1  = sim[pusher][dtw]["test_electrons"]["tracks"]["B1"][particle,:]
    B2  = sim[pusher][dtw]["test_electrons"]["tracks"]["B2"][particle,:]
    B3  = sim[pusher][dtw]["test_electrons"]["tracks"]["B3"][particle,:]
    Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

    x1 = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:]
    x2 = sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:]
    x3 = sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:]

    x_par = x1 * B1 / Bmag + x2 * B2 / Bmag + x3 * B3 / Bmag
    return x_par

def _unwrap_periodic(q, L):
    """
    Make a periodic 1D coordinate continuous by removing wrap-arounds.
    q : 1D array (positions vs time)
    L : domain length (e.g., 40)
    """
    q = np.asarray(q, dtype=float)
    if q.size <= 1:
        return q.copy()
    dq = np.diff(q)
    # number of box-crossings between samples (integer, positive or negative)
    k = np.round(dq / L)
    # cumulative correction to apply from each point onward
    corr = np.cumsum(k) * L
    q_unwrapped = q - np.concatenate(([0.0], corr))
    return q_unwrapped

def lin_fit(sim, quant, particle, titermin=0, titermax=-1, L=None):
    """
    If L is given (e.g., 40), the series is treated as periodic and unwrapped.
    """
    t = sim["test_electrons"]["tracks"]["t"][particle, titermin:titermax]
    q = sim["test_electrons"]["tracks"][quant][particle, titermin:titermax]

    if L is not None:
        # plt.plot(t, q)
        # plt.show()
        q = _unwrap_periodic(q, L)
        # plt.plot(t, q)
        # plt.show()

    slope, intercept, r_value, p_value, std_err = linregress(t, q)
    return slope, intercept, r_value, p_value, std_err

def horizontal_fit(sim, quant, particle, titermin=0, titermax=-1):
    # Extract the data
    q = sim["test_electrons"]["tracks"][quant][particle, titermin:titermax]
    
    # Fit a horizontal line: y = c
    c = np.mean(q)
    
    # Optional: residuals and fit quality
    residuals = q - c
    std_err = np.std(residuals)  # how much the data deviates from the horizontal line
    
    return c, std_err

def round_size(n):
    """Round n to 1 significant digit."""
    if n == 0:
        return 0
    return round(n, -int(math.floor(math.log10(n))))

def mean_rel_err_with_ar1(e):
    e = np.asarray(e)              # 1) Ensure e is a NumPy array (vector ops, speed, safety).
    N = e.size                     # 2) Number of time samples.

    if N < 2:                      # 3) If we have <2 points, we can't estimate variability.
        return float(e.mean()), np.nan

    em = e.mean()                  # 4) Sample mean: this is the point you will plot.
    x = e - em                     # 5) Mean-center the series (needed for variance & autocorr).
    s2 = x.var(ddof=1)             # 6) Sample variance of e_t (unbiased: ddof=1).

    # 7) Lag-1 autocorrelation ρ(1): similarity between consecutive samples.
    #    num = Σ (x_t * x_{t-1}); den = Σ (x_t^2). ρ(1) = num/den.
    num = np.dot(x[1:], x[:-1])
    den = np.dot(x, x)
    if (den == 0 or N < 3):
        rho1 = 0.0                 #    If constant or too short, assume no autocorrelation.
    else:
        rho1 = np.clip(num / den, -0.99, 0.99)  #    Clip for numerical stability.

    # 8) Effective sample size Neff for AR(1)-like correlation:
    #    With positive ρ(1), neighboring points carry redundant info, so Neff < N.
    Neff = N * (1 - rho1) / (1 + rho1)
    Neff = float(np.clip(Neff, 1.0, N))   #    Keep in [1, N] to avoid pathologies.

    # 9) Standard error of the mean using Neff (not naive N):
    #    SE( ē ) ≈ sqrt( s^2 / Neff ).
    se = np.sqrt(s2 / Neff)

    return float(em), float(se)


def circle_residuals(params, x, y):
    x_c, y_c, r = params
    return np.sqrt((x - x_c)**2 + (y - y_c)**2) - r


def gyroradius(sim, particle):
    x = sim["test_electrons"]["tracks"]["x1"][particle]
    y = sim["test_electrons"]["tracks"]["x2"][particle]
    center_estimate = x.mean(), y.mean(), max(x) - x.mean()
    result = optimize.least_squares(circle_residuals, center_estimate, args=(x, y))
    x_c, y_c, r = result.x

    # --- 1σ error bar for r ---
    J = result.jac                         # N×3 Jacobian at the solution
    res = result.fun                       # residuals (length N)
    dof = max(1, res.size - result.x.size) # N - 3, guard against <=0
    sigma2 = (res @ res) / dof             # residual variance
    JTJ = J.T @ J
    try:
        cov = np.linalg.inv(JTJ) * sigma2
    except np.linalg.LinAlgError:
        cov = np.linalg.pinv(JTJ) * sigma2  # fallback if nearly singular
    r_err = float(np.sqrt(cov[2, 2]))

    return r, r_err
    # print("J", np.shape(J))
    # print("res", np.shape(res))
    # print("dof", dof)
    # print("sigma2", sigma2)
    # print("JTJ", JTJ)
    # print("cov", cov)
    # print("r_err", r_err)
    # return r


In [ ]:
# Normalize axis to w_ce

def _set_scaled_formatter(axis, scale_factor, fmt=".2f"):
    axis.set_major_formatter(
        FuncFormatter(lambda v, pos: f"{v*scale_factor:{fmt}}")
    )

def scale_x_ax(scale_factor, fig, ax, label=r"$t[2\pi / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_xlabel(label)
    if lock_ticks:
        # freeze current tick positions
        ticks = ax.get_xticks()
        ax.xaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    return fig, ax

def scale_y_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_ylabel(label)
    if lock_ticks:
        ticks = ax.get_yticks()
        ax.yaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    return fig, ax

def scale_z_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_zlabel(label)
    if lock_ticks:
        ticks = ax.get_zticks()
        ax.zaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax

def scale_3d_axes(scale_factor, fig, ax, fmt=".2f"):
    ax.set_xlabel(r"$x_1[c / \Omega_e]$")
    ax.set_ylabel(r"$x_2[c / \Omega_e]$")
    ax.set_zlabel(r"$x_3[c / \Omega_e]$")
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax


In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, avrg_B, tmin = 0, tmax = 100000000000, markersize=1.8, color = None, linewidth=0.8, linestyle='-', unload=False, fig=None, ax=None):
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)

    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.0f}$".format(t[0] * avrg_B / 2.0 / np.pi) + r"$ [2\pi / \Omega_e]$")
        label2 = ("$t = {:.0f}$".format(t[-1] * avrg_B / 2.0 / np.pi)  + r"$[ 2\pi / \Omega_e]$")
        print("tmin, tmax traj: ", label1, label2)
        label1 = None
        label2 = None
        
 
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle=linestyle, markersize=markersize, linewidth=linewidth, label=label, color=color)
    
    # Labels and title
    fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax)
    ax.legend()

    return fig, ax

## Gyromotion 3d

In [ ]:
test = "Gyro"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Borisv2': ["1000", "100", "10", "1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sametimeIter ={
    "1000" : 10,
    "100" : 100,
    "10" : 1000,
    "1" : 10000,
    "0_1" : 10000,
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
particle = 0
avrg_B = 1000.0



# fig, ax = plt.subplots()
# ax.plot(baseline["test_electrons"]["tracks"]["t"][particle,:], 
#         baseline["test_electrons"]["tracks"]["x1"][particle,:], 
#         label = fr"Baseline", 
#         marker='o', linestyle='-', markersize=0, linewidth=0.8, color="black")

# # ax.set_xlim([0, 0.1])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B/2.0/np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

# ax.legend()

# plt.show()



# fig, ax = plt.subplots()
# ref = gyroradius(baseline, particle)
# ref = np.max(baseline["test_electrons"]["tracks"]["x2"][particle,:])
# for pusher in ["Boris"]:
#     for dtw in sim[pusher]:
#         ax.plot(ref-sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xscale('log')
# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B, label = r"$x_2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# Ro / error
fig, ax = plt.subplots()
ro_baseline, err_baseline = gyroradius(baseline, particle)
for pusher in ["Boris", "Borisv2"]:
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        if pusher == "Borisv2":
            x.append(float(dtw.replace("_", "."))/2.0/np.pi)
        else:
            x.append(float(dtw.replace("_", ".")))
        ro, stderr = gyroradius(sim[pusher][dtw], particle)
        # print("ro, stderr", ro, stderr)
        y.append(abs((ro - ro_baseline) / ro_baseline))
        err.append(np.sqrt((stderr / ro_baseline)**2 + (ro * err_baseline / ro_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )


ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$\rho$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.legend()
ax.grid()
plt.tight_layout()
plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p3
p_par_baseline = p_par_baseline[index_min:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1
        p_par = sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,:]
        p_par = p_par[index_min:sametimeIter[dtw]+1]

        if pusher == "Borisv2":
            x.append(float(dtw.replace("_", "."))/2.0/np.pi)
        else:
            x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[::step])/p_par_baseline[::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCca":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    y = np.array(y)
    y[y == 0] = 1e-18  # show them at the bottom of the log plot
    
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.legend()
ax.grid()
plt.tight_layout()
plt.show()


# v1 abs error
fig, ax = plt.subplots()
index_min = 1
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
        if pusher == "Borisv2":
            x.append(float(dtw.replace("_", "."))/2.0/np.pi)
        else:
            x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - 0.0)))
        err.append(np.sqrt((stderr)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    y = np.array(y)
    y[y == 0] = 1e-18  # show them at the bottom of the log plot

    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

index = baseline["test_electrons"]["tracks"]["t"].shape[1]
slope, intercept, _, _, stderr = lin_fit(baseline, "x1", particle, titermin=index_min, titermax=index)
ax.errorbar(
    0.01, abs((slope - 0.0)), yerr=np.sqrt((stderr)**2),
    label=fr"Baseline",
    color="black",
    fmt='o',
    linestyle='-',
    markersize=3.5,
    linewidth=1.2,
    capsize=3,
    elinewidth=1.0,
)
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_1$ absolute error $[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.legend()
ax.grid()
ax.set_ylim(None, 1e-7)
plt.tight_layout()
plt.show()



fig, ax = None, None
dtw = "10"
fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=28.274333882308149, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=28.274333882308149, fig=fig, ax=ax, markersize=0, linewidth=3.)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=28.274333882308149, fig=fig, ax=ax, markersize=0, linewidth=3., linestyle=(0, (4, 4)))
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1.5, tmax=28.274333882308149, fig=fig, ax=ax)

# ax.view_init(elev=15, azim=80)
# ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=5)
ax.tick_params(axis='z', pad=15)

# ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.legend()
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 1  

plt.show()

## Relativistic Gyromotion

In [ ]:
test = "GyroRel"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    # 'Borisv2': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sametimeIter ={
    "1000" : 10,
    "100" : 100,
    "10" : 1000,
    "1" : 10000,
    "0_1" : 10000,
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
particle = 0
avrg_B = 1000.0

# import matplotlib.pyplot as plt
# from matplotlib import cm
# dtw = "1000"
# pusher = "Borisv2"
# particle=0
# # Example setup (assuming `baseline` and `particle` are defined)
# x2 = sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle, :]
# x1 = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle, :]
# t_vals = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle, :]

# # Normalize colors to time
# norm = plt.Normalize(t_vals.min(), t_vals.max())
# colors = cm.viridis(norm(t_vals))

# # Create figure and axis
# fig, ax = plt.subplots()

# # Plot trajectory segment by segment
# for i in range(len(x2) - 1):
#     ax.plot(x2[i:i+2], x1[i:i+2], color=colors[i], linewidth=0, markersize=1, marker='o')

# # Add colorbar
# sm = plt.cm.ScalarMappable(cmap=cm.viridis, norm=norm)
# sm.set_array(t_vals)  # use the time values for colorbar
# cbar = plt.colorbar(sm, ax=ax, label="Time")

# # Labels
# ax.set_xlabel(r"$x_2$")
# ax.set_ylabel(r"$x_1$")

# plt.show()


# fig, ax = plt.subplots()
# ax.plot(baseline["test_electrons"]["tracks"]["t"][particle,:], 
#         baseline["test_electrons"]["tracks"]["x1"][particle,:], 
#         label = fr"Baseline", 
#         marker='o', linestyle='-', markersize=1, linewidth=0, color="black")
# dtw = "1"
# pusher = "Boris"
# ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#         sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#         label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#         marker='o', linestyle='-', markersize=1, linewidth=0)
# # dtw = "10"
# # pusher = "Borisv2"
# # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
# #         sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
# #         label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #         marker='o', linestyle='-', markersize=1, linewidth=0.8)
# # ax.set_xlim([0, 0.1])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B/2.0/np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

# ax.legend()

# plt.show()

# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B, label = r"$x_2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# Ro / error
fig, ax = plt.subplots()
ro_baseline, err_baseline = gyroradius(baseline, particle)
for pusher in ["Boris"]:
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        x.append(float(dtw.replace("_", ".")))
        ro, stderr = gyroradius(sim[pusher][dtw], particle)
        # print("ro, stderr", ro, stderr)
        y.append(abs((ro - ro_baseline) / ro_baseline))
        err.append(np.sqrt((stderr / ro_baseline)**2 + (ro * err_baseline / ro_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )


ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$\rho$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.legend()
ax.grid()
plt.tight_layout()
plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p3
p_par_baseline = p_par_baseline[index_min:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1
        p_par = sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,:]
        p_par = p_par[index_min:sametimeIter[dtw]+1]

        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[::step])/p_par_baseline[::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCca":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    y = np.array(y)
    y[y == 0] = 1e-18  # show them at the bottom of the log plot
    
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()
plt.show()


# v1 abs error
fig, ax = plt.subplots()
index_min = 1
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]
        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - 0.0)))
        err.append(np.sqrt((stderr)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    y = np.array(y)
    y[y == 0] = 1e-18  # show them at the bottom of the log plot

    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )
index = baseline["test_electrons"]["tracks"]["t"].shape[1]
slope, intercept, _, _, stderr = lin_fit(baseline, "x1", particle, titermin=index_min, titermax=index)
ax.errorbar(
    0.01, abs((slope - 0.0)), yerr=np.sqrt((stderr)**2),
    label=fr"Baseline",
    color="black",
    fmt='o',
    linestyle='-',
    markersize=3.5,
    linewidth=1.2,
    capsize=3,
    elinewidth=1.0,
)
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_1$ absolute error $[c]$")

ax.legend()
ax.grid()
plt.tight_layout()
plt.show()



fig, ax = None, None

dtw = "10"
tmax=3.07876080051799769
  
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=0.6, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3.)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3., linestyle=(0, (4, 4)))

# ax.view_init(elev=0, azim=80)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=10)

# ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))


fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
# ax.zaxis.labelpad = 1

ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(1.05, 0.54, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")

ax.legend()
plt.show()

## EXB 3d

In [ ]:
test = "EXB"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
    'Vay': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
# pusher = "Boris"
# dtw = "100"
# t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][0]
# x3 = sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][0]

# x3 = _unwrap_periodic(x3, 40)
# plt.plot(t[:], x3[:])
# plt.plot(t[:], 0.01 * t[:]+5, label="0.01 t")
# plt.legend()
# plt.show()

# x1 = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][0]
# x1 = _unwrap_periodic(x1, 40)
# plt.plot(t[:], x1[:])
# plt.plot(t[:], 0.001 * t[:]+5, label="0.001 t")
# plt.legend()
# plt.show()

In [ ]:
particle = 0
avrg_B = 1000.0

# P total
fig, ax = plt.subplots()
for pusher in ["Boris", "Vay"]:
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
                np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
                sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
                sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
                label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()




# # x3 / x1
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 0
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,index_min:index], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([5, 8.0])
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 np.sqrt(    
#                     sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,index_min:index]**2
#                     + sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,index_min:index]**2
#                     + sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,index_min:index]**2
#                     ), 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p3~ / t
# fig, ax = plt.subplots()
# for pusher in ["Boris", "Gca"]:
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p3", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p1~ / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v1~ / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]    

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# dtw="10"
# ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:] * 0.0 + 400./1000., 
#         label = "Theoretical", 
#         marker='o', linestyle='-', markersize=0.0, linewidth=0.6, color='black')
# ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v1~ / rel error
fig, ax = plt.subplots()
# fig2, ax2 = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x1", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    y2 = []
    err = []
    err2 = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index, L=40.0)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        y2.append(slope)
        err2.append(stderr)
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    elif pusher == "Vay":
        linestyle='dotted'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )
    # ax2.errorbar(
    #     x, y2, yerr=err2,
    #     label=fr"{pusher}",
    #     fmt='o',
    #     linestyle=linestyle,
    #     markersize=3.5,
    #     linewidth=1.2,
    #     capsize=3,
    #     elinewidth=1.0,
    # )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_E$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.legend()
ax.grid()
plt.tight_layout()
# ax2.set_yscale('log')
# ax2.set_xscale('log') 
# ax2.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax2.set_ylabel(r"$v_1$")
plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
# fig2, ax2 = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    y2 = []
    err = []
    err2 = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index, L=40.0)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        y2.append(slope)
        err2.append(stderr)
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )
    # ax2.errorbar(
    #     x, y2, yerr=err2,
    #     label=fr"{pusher}",
    #     fmt='o',
    #     linestyle=linestyle,
    #     markersize=3.5,
    #     linewidth=1.2,
    #     capsize=3,
    #     elinewidth=1.0,
    # )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_\parallel$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.legend()
ax.grid()
plt.tight_layout()

# ax2.set_yscale('log')
# ax2.set_xscale('log') 
# ax2.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax2.set_ylabel(r"$v_3$")
plt.show()



# 3D Trajectory
fig, ax = None, None
# tmax=3.07876080051799769
tmax=62.831853071795861
dtw = "100"

fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3.5)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3.5, linestyle=(0, (4, 4)))
fig, ax = plot_trajectory(sim["Vay"][dtw], fr"Vay", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1.2, tmax=tmax, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-85)
# ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

# ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.legend()
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 30
# ax.zaxis.labelpad = 1
ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(0.95, 0.54, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()


## Relativistic $E\times B$ drift

In [ ]:
test = "EXBRel"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
    'Vay': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
# pusher = "Boris"
# dtw = "0_1"
# # t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][0]
# # x3 = sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][0]

# # x3 = _unwrap_periodic(x3, 40)
# # plt.plot(t[:], x3[:])
# # plt.plot(t[:], 0.7/np.sqrt(1+3*0.7**2) * t[:]+5, label="0.7 t")
# # plt.legend()
# # plt.show()

# for dtw in sim[pusher]:
#     t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][0]
#     x1 = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][0]
#     # x1 = _unwrap_periodic(x1, 40)
#     plt.plot(t[:], x1[:], label = fr"$\Delta t\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$")
# plt.plot(t[:], 0.4 * t[:]+5, label="0.4 t")
# plt.plot(t[:], 0.8 * t[:]+5, label="0.8 t")
# plt.legend()
# plt.xlim([0, 62])
# plt.ylim([0, 62])
# plt.show()

In [ ]:
particle = 0
avrg_B = 1000.0

# # x3 / x1
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1])
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,index_min:index], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p3~ / t
# fig, ax = plt.subplots()
# for pusher in ["Boris"]:
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1])

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p3", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p1~ / t
# fig, ax = plt.subplots()
# for pusher in ["Vay", "Gca"]:
#     for dtw in ["1000", "100", "10"]:
#         index_min = 1
#         index = -1

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index, L=40.0)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
# slope, intercept, _, _, _ = lin_fit(baseline, "p1", particle, titermin=index_min, titermax=index, L=40.0)
# ax.plot(baseline["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#         baseline["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#         label = fr"Baseline", 
#         marker='o', linestyle='-', markersize=1.5, linewidth=0.8, color='black')
# ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))
# ax.set_xlim([0, 62])
# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v1~ / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
        # index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1])

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# dtw="10"
# ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:] * 0.0 + 400./1000., 
#         label = "Theoretical", 
#         marker='o', linestyle='-', markersize=0.0, linewidth=0.6, color='black')

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# P total
fig, ax = plt.subplots()
for pusher in ["Boris", "Vay"]:
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
                np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
                sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
                sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
                label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p3
p_par_baseline = p_par_baseline[index_min:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1
        p_par = sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,:]
        p_par = p_par[index_min:sametimeIter[dtw]+1]

        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[::step])/p_par_baseline[::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCca":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    y = np.array(y)
    y[y == 0] = 1e-18  # show them at the bottom of the log plot
    
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()
plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
# fig2, ax2 = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    y2 = []
    err = []
    err2 = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index, L=40.0)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        y2.append(slope)
        err2.append(stderr)
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )
    # ax2.errorbar(
    #     x, y2, yerr=err2,
    #     label=fr"{pusher}",
    #     fmt='o',
    #     linestyle=linestyle,
    #     markersize=3.5,
    #     linewidth=1.2,
    #     capsize=3,
    #     elinewidth=1.0,
    # )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_\parallel$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.legend()
ax.grid()
plt.tight_layout()

# ax2.set_yscale('log')
# ax2.set_xscale('log') 
# ax2.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax2.set_ylabel(r"$v_3$")
plt.show()



# v1~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x1", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = next((i for i, x in enumerate(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,index_min:]) if ((x > 40) or (x < 5))), sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1])
        index = -1
 
        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index, L=40.0)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    elif pusher == "Vay":
        linestyle='dotted'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )


ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_E$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()

plt.show()



# 3D Trajectory
fig, ax = None, None

dtw = "10"
tmax=2.01061930000001

fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3.5)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3.5, linestyle=(0, (4, 4)))
fig, ax = plot_trajectory(sim["Vay"][dtw], fr"Vay", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1., tmax=tmax, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend()
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(1.02, 0.62, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()


## $\nabla B$ Drift

In [ ]:
test = "GradB"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sametimeIter ={
    "1000" : 10,
    "100" : 100,
    "10" : 1000,
    "1" : 10000,
    "0_1" : 10000,
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
# boris1000short = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw1000_short/Boris.in")
# baseline = sim["Gca"]["10"]
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
# pusher = "Boris"
# dtw = "0_1"
# # t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][0]
# # x3 = sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][0]

# # x3 = _unwrap_periodic(x3, 40)
# # plt.plot(t[:], x3[:])
# # plt.plot(t[:], 0.7/np.sqrt(1+3*0.7**2) * t[:]+5, label="0.7 t")
# # plt.legend()
# # plt.show()

# for dtw in ["1", "0_1"]:
#     t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][0]
#     x1 = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][0]
#     x1 = _unwrap_periodic(x1, 40)
#     plt.plot(t[:], x1[:], label = fr"$\Delta t\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$")


# t = sim[pusher]["1000"]["test_electrons"]["tracks"]["t"][0]
# plt.plot(t[:], 3.9e-6 * t[:]+5.25, label="theo")
# # plt.plot(t[:], 0.8 * t[:]+5, label="0.8 t")
# plt.legend()
# plt.xlim([0, 62])
# # plt.ylim([0, 62])
# plt.show()

In [ ]:
particle = 0
avrg_B = 10.0
# avrg_B = np.average(np.sqrt(baseline["test_electrons"]["tracks"]["B1"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B2"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B3"][particle,:]**2))
# print("avrg_B:", avrg_B)

# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])
# # ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# ax.set_ylabel(r"$p_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p1~ / rel error
# fig, ax = plt.subplots()
# fig2, ax2 = plt.subplots()
# index_min = 1
# base, base_Err = horizontal_fit(baseline, "p1", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
# for pusher in sim.keys():
#     x = []
#     y = []
#     y2 = []
#     err = []
#     err2 = []
#     for dtw in sim[pusher]:
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]
#         val, val_Err = horizontal_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index)

#         x.append(float(dtw.replace("_", ".")))
#         y.append(abs((val - base) / base))
#         y2.append(val)
#         err2.append(val_Err)
#         err.append(np.sqrt((val_Err / base)**2 + (val * base_Err / base**2)**2))
#     if pusher == "GcaCorr":
#         linestyle=(0, (4, 4))
#     elif pusher == "Vay":
#         linestyle='dotted'
#     else:
#         linestyle='-'
#     ax.errorbar(
#         x, y, yerr=err,
#         label=fr"{pusher}",
#         fmt='o',
#         linestyle=linestyle,
#         markersize=3.5,
#         linewidth=1.2,
#         capsize=3,
#         elinewidth=1.0,
#     )
#     ax2.errorbar(
#         x, y2, yerr=err2,
#         label=fr"{pusher}",
#         fmt='o',
#         linestyle=linestyle,
#         markersize=3.5,
#         linewidth=1.2,
#         capsize=3,
#         elinewidth=1.0,
#     )

# ax.set_yscale('log')
# ax.set_xscale('log') 
# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax.set_ylabel(r"$p_1$ relative error")

# # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# ax.legend()
# ax.grid()
# plt.tight_layout()
# ax2.set_yscale('log')
# ax2.set_xscale('log') 
# ax2.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax2.set_ylabel(r"$p_1$")
# plt.show()



# # v1~ / t
# fig, ax = plt.subplots()
# for pusher in ["Gca"]:
#     for dtw in ["1000"]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_yscale('log')
# ax.set_xlim([0, 6283.185307179586476925286766559])
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v1~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x1", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_{\nabla B}$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()

plt.show()



# 3D Trajectory
fig, ax = None, None

dtw = "10"
tmax = 6283.185307179586476925286766559


# Boris1000short = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw1000short/Boris.in")

fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1.6, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3.)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3., linestyle=(0, (4, 4)))

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=5)
ax.tick_params(axis='z', pad=15)

ax.legend()
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.set_xlim(-15, 15) 
# ax.set_ylim(-15, 15)
# ax.set_zlim(6.4, 6.6)

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(1.01, 0.59, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()



## Relativistic $\nabla B$ Drift

In [ ]:
test = "GradBRel_v2"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sametimeIter ={
    "1000" : 10,
    "100" : 100,
    "10" : 1000,
    "1" : 10000,
    "0_1" : 10000,
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
# baseline = sim["Gca"]["10"]
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
# pusher = "Boris"
# dtw = "0_1"
# # t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][0]
# # x3 = sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][0]

# # x3 = _unwrap_periodic(x3, 40)
# # plt.plot(t[:], x3[:])
# # plt.plot(t[:], 0.7/np.sqrt(1+3*0.7**2) * t[:]+5, label="0.7 t")
# # plt.legend()
# # plt.show()

# for dtw in ["100"]:
#     t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][0]
#     x1 = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][0]
#     # x1 = _unwrap_periodic(x1, 40)
#     plt.plot(t[:], x1[:], label = fr"$\Delta t\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$")
# t = sim[pusher]["100"]["test_electrons"]["tracks"]["t"][0]
# plt.plot(t[:], 3.65413e-5 * t[:]+5.245, label="theo")
# # plt.plot(t[:], 0.8 * t[:]+5, label="0.8 t")
# plt.legend()
# # plt.xlim([0, 62])
# # plt.ylim([0, 62])
# plt.show()

In [ ]:
particle = 0
avrg_B = 1000.0
# avrg_B = np.average(np.sqrt(baseline["test_electrons"]["tracks"]["B1"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B2"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B3"][particle,:]**2))
# print("avrg_B:", avrg_B)

# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])
# # ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "p1", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * slope + intercept, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# ax.set_ylabel(r"$p_1[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v1~ / t
# fig, ax = plt.subplots()
# for pusher in ["Gca"]:
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
#         slope = -slope
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
# 
# ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_1[c]$")
# 
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# 
# plt.show()



# v1~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x1", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x1", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_{\nabla B}$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()

plt.show()



# 3D Trajectory
fig, ax = None, None

dtw = "10"
# tmax = 62.83185307179586476925286766559
tmax = 62.831853079

# Boris1000short = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw1000short/Boris.in")

fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=.6, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3.)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=3., linestyle=(0, (4, 4)))

ax.view_init(elev=10, azim=-80)
# ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=10)
ax.tick_params(axis='z', pad=15)

ax.legend()
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.set_xlim(-15, 15) 
# ax.set_ylim(-15, 15)
# ax.set_zlim(6.4, 6.6)

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 30 

ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(0.96, 0.62, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()

## Curvature Drift

In [ ]:
test = "Curv_v2"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sametimeIter ={
    "1000" : 20,
    "100" : 200,
    "10" : 2000,
    "1" : 20000,
    "0_1" : 20000,
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
# baseline = sim["Gca"]["10"]
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in ["1000"]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:sametimeIter[dtw]+1], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:sametimeIter[dtw]+1], 
                label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=0, linewidth=0.8)

ax.set_xlim([-15, 15])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x_2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
r = sim["Gca"]["10"]["test_electrons"]["tracks"]["x1"][0]**2 + sim["Gca"]["10"]["test_electrons"]["tracks"]["x2"][0]**2 + sim["Gca"]["10"]["test_electrons"]["tracks"]["x3"][0]**2
r = np.sqrt(r)

print(r_mean:=np.mean(r), np.std(r))

p_par=p_parallel(sim, "Gca", "10", 0)

print(p_par_mean:=np.mean(p_par), np.std(p_par))

print(dtw_curv := r_mean / UtoV(abs(p_par_mean)) )

dtw_curv * avrg_B / 2.0 / np.pi

In [ ]:
particle = 0
avrg_B = 1000.0

# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()


# # P parallel
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,0:], 
#                 p_parallel(sim, pusher, dtw, particle)[0:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_\parallel[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        p_par = p_parallel(sim, pusher, dtw, particle)
        p_par = p_par[index_min:sametimeIter[dtw]+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )
ax.vlines(dtw_curv * avrg_B / 2.0 / np.pi, ymin=1e-8, ymax=1e-1, colors='gray', linestyles='dashed', label=r"$L_c / v_i^{GC}$")
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()

plt.show()




# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2. / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# fig, ax = plt.subplots()
# for pusher in ["Gca"]:
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v3~ / t
# fig, ax = plt.subplots()
# for pusher in ["Gca"]:
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p3 / t
# fig, ax = plt.subplots()
# for pusher in ["Gca"]:
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,index_min:index], 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))

    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.vlines(dtw_curv * avrg_B / 2.0 / np.pi, ymin=1e-7, ymax=1, colors='gray', linestyles='dashed', label=r"$L_c / v_i^{GC}$")
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_c$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()

plt.show()



# 3D Trajectory
fig, ax = None, None

dtw = "1000"
tmax=125.663706143591729
# tmax=1000000000000
fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, linewidth=2.5)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, linewidth=1.5)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1., tmax=tmax, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-190)
ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=0)
ax.tick_params(axis='z', pad=15)

ax.legend()
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.set_xlim(-15, 15) 
# ax.set_ylim(-15, 15)
# ax.set_zlim(6.4, 6.6)

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 30
ax.yaxis.labelpad = 5
ax.zaxis.labelpad = 30 

ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(1.01, 0.49, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()


## Relative Curvature Drift

In [ ]:
test = "CurvRel_v2"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sametimeIter ={
    "1000" : 20,
    "100" : 200,
    "10" : 2000,
    "1" : 20000,
    "0_1" : 20000,
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])

In [ ]:
r = sim["Gca"]["10"]["test_electrons"]["tracks"]["x1"][0]**2 + sim["Gca"]["10"]["test_electrons"]["tracks"]["x2"][0]**2 + sim["Gca"]["10"]["test_electrons"]["tracks"]["x3"][0]**2
r = np.sqrt(r)

print(r_mean:=np.mean(r), np.std(r))

p_par=p_parallel(sim, "Gca", "10", 0)

print(p_par_mean:=np.mean(p_par), np.std(p_par))

print(dtw_curv := r_mean / UtoV(abs(p_par_mean)) )

dtw_curv * avrg_B / 2.0 / np.pi

In [ ]:
particle = 0
avrg_B = 1000.0

# # fig, ax = plt.subplots()
# # for pusher in ["Gca"]:
# #     for dtw in ["10"]:
# #         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
# #                 sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
# #                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #                 marker='o', linestyle='-', markersize=0, linewidth=0.8)

# # ax.set_xlim([-15, 15])

# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B, label = r"$x_1[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
# # fig, ax = scale_y_ax(avrg_B, label = r"$x_2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".1f")

# # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# # plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        p_par = p_parallel(sim, pusher, dtw, particle)
        p_par = p_par[index_min:sametimeIter[dtw]+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )
ax.vlines(dtw_curv * avrg_B / 2.0 / np.pi, ymin=1e-5, ymax=5, colors='gray', linestyles='dashed', label=r"$L_c / v_i^{GC}$")
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()

plt.show()




# # fig, ax = plt.subplots()
# # for pusher in ["Gca"]:
# #     for dtw in sim[pusher]:
# #         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
# #                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:], 
# #                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # # ax.set_xlim([0, 40])

# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# # ax.set_ylabel(r"$p_3[c]$")

# # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# # plt.show()



# # v3~ / t
# fig, ax = plt.subplots()
# for pusher in ["Gca"]:
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # # P total
# # fig, ax = plt.subplots()
# # for pusher in ["Gca"]:
# #     for dtw in ["1000"]:
# #         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
# #                 np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
# #                 sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
# #                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
# #                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # # ax.set_xlim([0, 40])

# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# # ax.set_ylabel(r"$p[c]$")

# # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# # plt.show()



# # P parallel
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in ["1000"]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#                 p_parallel(sim, pusher, dtw, particle)[2:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_\parallel[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))

    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.vlines(dtw_curv * avrg_B / 2.0 / np.pi, ymin=1e-6, ymax=1e2, colors='gray', linestyles='dashed', label=r"$L_c / v_i^{GC}$")
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))
# ax.set_xlim(None,3000)
ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_c$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()
plt.show()




# 3D Trajectory
fig, ax = None, None

dtw = "1000"
tmax=125.663706143591729

fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmax=55, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, linewidth=2.5)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmax=tmax, fig=fig, ax=ax, linewidth=1.5)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1., tmax=tmax, fig=fig, ax=ax)

ax.view_init(elev=35, azim=-70)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend()
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

ax.set_xlim(-15, 15) 
ax.set_ylim(-15, 15)
# ax.set_zlim(6.4, 6.6)

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad =5
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 30 

ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(1.02, 0.63, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()


## $E_\parallel$ and $v_E \cdot ( b \cdot \nabla) B$ Forces

In [ ]:
test = "EGradB"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.



# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#                 label = fr"{pusher}, x_1, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
#                 label = fr"{pusher}, x_2, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, x_3, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # P total
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#                 np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v3~ / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# P parallel
fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
                p_parallel(sim, pusher, dtw, particle)[1:], 
                label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_\parallel[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        p_par = p_parallel(sim, pusher, dtw, particle)
        p_par = p_par[index_min:round_size(len(p_par))+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))

    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        # linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_3$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
for pusher in sim.keys():
    for dtw in ["10"]:
        fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"]["1000"], fr"Gca, $\Delta t\, \Omega_e / 2\pi = 1000$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=0.6, tmax=50000000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=10)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 30 

plt.show()

## Relativistic $E_\parallel$ and $v_E \cdot ( b \cdot \nabla) B$ Forces

In [ ]:
test = "EGradBRel"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.



# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#                 label = fr"{pusher}, x_1, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
#                 label = fr"{pusher}, x_2, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, x_3, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # P total
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#                 np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # v3~ / t
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         index_min = 1
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

#         slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
#                 label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_yscale('log')
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$v_3[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# P parallel
fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
                p_parallel(sim, pusher, dtw, particle)[1:], 
                label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_\parallel[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        p_par = p_parallel(sim, pusher, dtw, particle)
        p_par = p_par[index_min:round_size(len(p_par))+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))

    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        # linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_3$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
for pusher in sim.keys():
    for dtw in ["10"]:
        fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
# fig, ax = plot_trajectory(sim["Gca"]["1000"], fr"Gca, $\Delta t\, \Omega_e / 2\pi = 1000$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=0.6, tmax=50000000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=5)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(3))
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 25 

plt.show()

## $( v_E \cdot \nabla) b$

In [ ]:
test = "PerpEGradB"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.



# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#                 label = fr"{pusher}, x_1, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
#                 label = fr"{pusher}, x_2, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, x_3, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # P total
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#                 np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v3~ / t
fig, ax = plt.subplots()
for pusher in ["Gca"]:
    for dtw in sim[pusher]:
        index_min = 1
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
                label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.set_yscale('log')
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$v_3[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# P parallel
fig, ax = plt.subplots()
for pusher in sim.keys():
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
                p_parallel(sim, pusher, dtw, particle)[1:], 
                label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_\parallel[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        p_par = p_parallel(sim, pusher, dtw, particle)
        p_par = p_par[index_min:round_size(len(p_par))+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))

    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        # linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_3$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
pusher = "Gca"
ftw = "1000"

fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
# fig, ax = plot_trajectory(sim["Gca"]["1000"], fr"Gca, $\Delta t\, \Omega_e / 2\pi = 1000$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=0.6, tmax=50000000, fig=fig, ax=ax)

ax.view_init(elev=15, azim=-70)
# ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=10)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".1f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 30 

plt.show()

## Relativistic $( v_E \cdot \nabla) b$

In [ ]:
test = "PerpEGradBRel"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sametimeIter ={
    "1000" : 10,
    "100" : 100,
    "10" : 1000,
    "1" : 10000,
    "0_1" : 10000,
}
    
sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.



# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#         #         sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:], 
#         #         label = fr"{pusher}, x_1, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#         #         sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:], 
#         #         label = fr"{pusher}, x_2, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:], 
#                 label = fr"{pusher}, x_3, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # P total
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#                 np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# v3~ / t
fig, ax = plt.subplots()
for pusher in ["Gca"]:
    for dtw in sim[pusher]:
        index_min = 1
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, _ = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index], 
                sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:index] * 0.0 + slope, 
                label = fr"{pusher}, fit, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.set_yscale('log')
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$v_3[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# P parallel
fig, ax = plt.subplots()
for pusher in ["Gca"]:
    for dtw in sim[pusher]:
        ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
                p_parallel(sim, pusher, dtw, particle)[1:], 
                label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
                marker='o', linestyle='-', markersize=0, linewidth=0.8)
        
B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag

ax.plot(baseline["test_electrons"]["tracks"]["t"][particle,1:], 
        p_par[1:], 
        label = fr"Baseline", 
        marker='o', linestyle='-', markersize=0, linewidth=0.8, color="black")
# ax.set_xlim([0, 40])

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
ax.set_ylabel(r"$p_\parallel[c]$")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# p_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        p_par = p_parallel(sim, pusher, dtw, particle)
        p_par = p_par[index_min:round_size(len(p_par))+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$p_\parallel$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



# v3~ / rel error
fig, ax = plt.subplots()
index_min = 1
slope_baseline, intercept_baseline, _, _, err_baseline = lin_fit(baseline, "x3", particle, titermin=index_min, titermax=baseline["test_electrons"]["tracks"]["t"].shape[1])
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]

        slope, intercept, _, _, stderr = lin_fit(sim[pusher][dtw], "x3", particle, titermin=index_min, titermax=index)
        x.append(float(dtw.replace("_", ".")))
        y.append(abs((slope - slope_baseline) / slope_baseline))
        err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))

    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
        # linestyle='-'
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$v_3$ relative error")

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()



fig, ax = None, None
for pusher in sim.keys():
    for dtw in ["1000"]:
        fig, ax = plot_trajectory(sim[pusher][dtw], fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw}$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
# fig, ax = plot_trajectory(sim["Gca"]["1000"], fr"Gca, $\Delta t\, \Omega_e / 2\pi = 1000$", particle=particle, avrg_B=avrg_B, tmax=50000000, fig=fig, ax=ax)
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=0.6, tmax=50000000, fig=fig, ax=ax)

ax.view_init(elev=5, azim=-80)
ax.tick_params(axis='x', pad=5)
ax.tick_params(axis='y', pad=10)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

# ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 30 

plt.show()

## Mirror Force

In [ ]:
test = "Mirror"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])


# tmax = 91.106186959
# tmin = 40.2123859
# sim["Gca"]["100"]["test_electrons"]["tracks"]["t"][0]

In [ ]:
particle = 0
# avrg_B = np.average(np.sqrt(baseline["test_electrons"]["tracks"]["B1"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B2"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B3"][particle,:]**2))
# print("avrg_B:", avrg_B)
avrg_B = 1000.
tmin = 7
tmax = 100000000000

sim_gca = sim["Gca"]["1000"]

gradB=get_gradB(sim_gca)
interp_gradB_1 = diag_interpolator(gradB[0], t=0)
interp_gradB_2 = diag_interpolator(gradB[1], t=0)
interp_gradB_3 = diag_interpolator(gradB[2], t=0)

J_b = get_Jacobian_b(sim_gca)
J_b_Interp = get_Jacobian_b_Interp(J_b, t=0)
sim_gca = sim["Gca"]["10"]
fields = call_silently(get_fields, sim_gca, particle, tmin, tmax)
traj = call_silently(get_trajectory, sim_gca, particle, tmin, tmax)
p = call_silently(get_p, sim_gca, particle, tmin, tmax)

E = fields[0:3, :]
B = fields[3:6, :]
B_norm = np.linalg.norm(B, axis=0)
b = B / B_norm
b_B = b / B_norm
t = fields[-1, :]
dt = t[1]-t[0] 

gradB_traj = np.stack([
    interp_gradB_1(traj[0:3].T),
    interp_gradB_2(traj[0:3].T),
    interp_gradB_3(traj[0:3].T)
], axis=0)

bGradB_traj = np.einsum('it,it->t', b, gradB_traj)


J_b_traj = np.empty((3, 3, np.shape(t)[0]), dtype=float)
for i in range(3):
    for j in range(3):
        J_b_traj[i,j] = J_b_Interp[i,j](traj[0:3].T)

curvature = np.einsum('tj,tij->it', b.T, J_b_traj.transpose(2, 0, 1))

# plt.plot(t, B_norm)

L_mirror = (np.max(B_norm) - B_norm[2]) / np.max(bGradB_traj)

p0gca = np.sqrt(p[0, 2]**2 + p[1, 2]**2 + p[2, 2]**2)
dtp = dt*100 * p0gca
print(r"\Delta t v_gca_0 = ", dtp, "L_mirror =", L_mirror)
print("dtw_mirror", dtw_mirror := (L_mirror / p0gca))
print("dtw_mirror/2/pi", (L_mirror / p0gca * 2.0 * np.pi))
print(dt*100 )
# plt.plot(t, 1./bGradB_traj)
# plt.title("1 / (b . grad B)")
# plt.show()

# plt.plot(t, (np.max(B_norm)-B_norm[2])/bGradB_traj)
# plt.title("Delta B / (b . grad B)")
# plt.show()

# plt.plot(t, B_norm/bGradB_traj)
# plt.title("B / (b . grad B)")
# plt.show()

# plt.plot(t, B_norm/np.linalg.norm(gradB_traj, axis=0))
# plt.title("B / (grad B)")
# plt.show()

# plt.plot(t, 1.0/np.linalg.norm(curvature, axis=0))
# plt.title("1 / (b . grad) b")
# plt.show()

# plt.plot(t, \Delta t * np.sqrt(p[1]**2 + p[2]**2 + p[0]**2))
# plt.title("\Delta t p")
# plt.show()


In [ ]:
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         if pusher == "GcaCorr":
#             linestyle=(0, (12, 12))
#         else:
#             linestyle='-'
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 x_parallel(sim, pusher, dtw, particle), 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle=linestyle, markersize=0, linewidth=1.)# markersize=1.5

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_\parallel[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# fig, ax = plt.subplots()
# for pusher in ["Gca"]:
#     for dtw in ["100", "1000"]:
#         B1  = sim[pusher][dtw]["test_electrons"]["tracks"]["B1"][particle,2:]
#         B2  = sim[pusher][dtw]["test_electrons"]["tracks"]["B2"][particle,2:]
#         B3  = sim[pusher][dtw]["test_electrons"]["tracks"]["B3"][particle,2:]
#         Bmag = np.sqrt(B1**2 + B2**2 + B3**2)
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:] * B1/Bmag + 
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:] * B2/Bmag +
#                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:] * B3/Bmag,
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
#         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         #         np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
#         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
#         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
#         #         label = fr"{pusher}, p, $\Delta t\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$", 
#         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
#         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:], 
#         #         label = fr"{pusher}, $ p_1, \Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:], 
#         #         label = fr"{pusher}, $ p_2, \Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
#         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
#         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:], 
#         #         label = fr"{pusher}, $ p_3, \Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
# # --- B on its own axis (right), same color as its line ---
# ax2 = ax.twinx()

# tB  = sim["Gca"]["100"]["test_electrons"]["tracks"]["t"][particle,2:]
# B1  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B1"][particle,2:]
# B2  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B2"][particle,2:]
# B3  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B3"][particle,2:]
# Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

# lnB, = ax2.plot(tB, Bmag, label=r"$B$", marker='o', linestyle='-',
#                 markersize=1.5, linewidth=0.8, color='purple')

# cB = lnB.get_color()
# ax2.spines["right"].set_color(cB)
# ax2.tick_params(axis='y', colors=cB)
# ax2.yaxis.label.set_color(cB)
# ax2.set_ylabel(r"$B$")

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))
# ax2.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_\parallel[c]$")

# # legend including B
# h1, l1 = ax.get_legend_handles_labels()
# h2, l2 = ax2.get_legend_handles_labels()
# ax.legend(h1 + h2, l1 + l2, loc='center left', bbox_to_anchor=(1, 0.5))
# ax.grid()

# plt.show()



# # P parallel
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
#                 p_parallel(sim, pusher, dtw, particle)[1:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_\parallel[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # p_parallel / rel error
# fig, ax = plt.subplots()
# index_min = 1

# B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
# B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
# B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
# Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

# p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
# p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
# p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

# p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
# p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
# t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
# for pusher in sim.keys():
#     x = []
#     y = []
#     err = []
#     for dtw in sim[pusher]:
#         step = int(float(dtw.replace("_", ".")))
#         if step < 1:
#             step = 1

#         p_par = p_parallel(sim, pusher, dtw, particle)
#         p_par = p_par[index_min:round_size(len(p_par))+1]
#         t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
#         x.append(float(dtw.replace("_", ".")))
#         em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
#         y.append(em)
#         #error with autocorrelation
#         err.append(se)
#     if pusher == "GcaCorr":
#         linestyle=(0, (4, 4))
#     else:
#         linestyle='-'
#     ax.errorbar(
#         x, y, yerr=err,
#         label=fr"{pusher}",
#         fmt='o',
#         linestyle=linestyle,
#         markersize=3.5,
#         linewidth=1.2,
#         capsize=3,
#         elinewidth=1.0,
#     )

# ax.set_yscale('log')
# ax.set_xscale('log') 
# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax.set_ylabel(r"$p_\parallel$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()




# x_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

x1 = baseline["test_electrons"]["tracks"]["x1"][particle,:]
x2 = baseline["test_electrons"]["tracks"]["x2"][particle,:]
x3 = baseline["test_electrons"]["tracks"]["x3"][particle,:]

x_par_baseline = x1 * B1 / Bmag + x2 * B2 / Bmag + x3 * B3 / Bmag
x_par_baseline = x_par_baseline[:round_size(len(x_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(x_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        x_par = x_parallel(sim, pusher, dtw, particle)
        x_par = x_par[index_min:round_size(len(x_par))+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(x_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((x_par - x_par_baseline[step::step])/x_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )
ax.vlines(dtw_mirror * avrg_B / 2.0 / np.pi, ymin=1e-6, ymax=5e-2, colors='gray', linestyles='dashed', label=r"$L_M / v_i^{GC}$")
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$x_\parallel$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()

plt.show()





fig, ax = None, None
# tmax = 81.681408993334639
# tmin = 56.548667764616289

tmax = 91.106186959
tmin = 40.2123859

dtw = "100"

fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmin=tmin, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmin=tmin, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=4.5)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmin=tmin, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=4.5, linestyle=(0, (3, 3)))
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1.0, tmin=tmin, tmax=tmax, fig=fig, ax=ax)

ax.view_init(elev=10, azim=-70)
# ax.view_init(elev=0, azim=270)
# ax.tick_params(axis='x', pad=15)
# ax.tick_params(axis='y', pad=15)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='upper left')
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 1 
ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(1.02, 0.52, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()

## Relativistic Mirror Force

In [ ]:
test = "MirrorRel"
dtw = "0_01"

sim_labels = {
    'Boris': ["1000", "100", "10", "1", "0_1"],
    'Gca': ["1000", "100", "10"],
    'GcaCorr': ["1000", "100", "10"],
}

sim = createSimDic(sim_labels, test)
baseline = ou.Simulation(f"/home/exxxx5/Tese/Decks/MethodicTests/{test}/Boris/dtw{dtw}/Boris.in")
print(baseline["test_electrons"]["tracks"]["x1"][0, 0], baseline["test_electrons"]["tracks"]["x2"][0, 0], baseline["test_electrons"]["tracks"]["x3"][0, 0])


In [ ]:
particle = 0
# avrg_B = np.average(np.sqrt(baseline["test_electrons"]["tracks"]["B1"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B2"][particle,:]**2 + baseline["test_electrons"]["tracks"]["B3"][particle,:]**2))
# print("avrg_B:", avrg_B)
avrg_B = 1000.
tmin = 7
tmax = 100000000000

sim_gca = sim["Gca"]["1000"]

gradB=get_gradB(sim_gca)
interp_gradB_1 = diag_interpolator(gradB[0], t=0)
interp_gradB_2 = diag_interpolator(gradB[1], t=0)
interp_gradB_3 = diag_interpolator(gradB[2], t=0)

J_b = get_Jacobian_b(sim_gca)
J_b_Interp = get_Jacobian_b_Interp(J_b, t=0)
sim_gca = sim["Gca"]["10"]
fields = call_silently(get_fields, sim_gca, particle, tmin, tmax)
traj = call_silently(get_trajectory, sim_gca, particle, tmin, tmax)
p = call_silently(get_p, sim_gca, particle, tmin, tmax)

E = fields[0:3, :]
B = fields[3:6, :]
B_norm = np.linalg.norm(B, axis=0)
b = B / B_norm
b_B = b / B_norm
t = fields[-1, :]
dt = t[1]-t[0]

gradB_traj = np.stack([
    interp_gradB_1(traj[0:3].T),
    interp_gradB_2(traj[0:3].T),
    interp_gradB_3(traj[0:3].T)
], axis=0)

bGradB_traj = np.einsum('it,it->t', b, gradB_traj)


J_b_traj = np.empty((3, 3, np.shape(t)[0]), dtype=float)
for i in range(3):
    for j in range(3):
        J_b_traj[i,j] = J_b_Interp[i,j](traj[0:3].T)

curvature = np.einsum('tj,tij->it', b.T, J_b_traj.transpose(2, 0, 1))

# plt.plot(t, B_norm)

L_mirror = (np.max(B_norm) - B_norm[2]) / np.max(bGradB_traj)

p0gca = np.sqrt(p[0, 2]**2 + p[1, 2]**2 + p[2, 2]**2)
dtp = dt*100 * p0gca
print(r"\Delta t v_gca_0 = ", dtp, "L_mirror =", L_mirror)
print("dtw_mirror", dtw_mirror := (L_mirror / p0gca))
print("dtw_mirror/2/pi", (L_mirror / p0gca * 2.0 * np.pi))
print(dt*100 )
# plt.plot(t, 1./bGradB_traj)
# plt.title("1 / (b . grad B)")
# plt.show()

# plt.plot(t, (np.max(B_norm)-B_norm[2])/bGradB_traj)
# plt.title("Delta B / (b . grad B)")
# plt.show()

# plt.plot(t, B_norm/bGradB_traj)
# plt.title("B / (b . grad B)")
# plt.show()

# plt.plot(t, B_norm/np.linalg.norm(gradB_traj, axis=0))
# plt.title("B / (grad B)")
# plt.show()

# plt.plot(t, 1.0/np.linalg.norm(curvature, axis=0))
# plt.title("1 / (b . grad) b")
# plt.show()

# plt.plot(t, \Delta t * np.sqrt(p[1]**2 + p[2]**2 + p[0]**2))
# plt.title("\Delta t p")
# plt.show()


In [ ]:
# fig, ax = plt.subplots()
# for pusher in  ["Boris", "Gca"]:
#     for dtw in sim[pusher]:
#         if pusher == "GcaCorr":
#             linestyle=(0, (12, 12))
#         else:
#             linestyle='-'
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 x_parallel(sim, pusher, dtw, particle), 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle=linestyle, markersize=0, linewidth=1.)# markersize=1.5
        

# B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
# B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
# B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
# Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

# x1 = baseline["test_electrons"]["tracks"]["x1"][particle,:]
# x2 = baseline["test_electrons"]["tracks"]["x2"][particle,:]
# x3 = baseline["test_electrons"]["tracks"]["x3"][particle,:]

# x_par_baseline = x1 * B1 / Bmag + x2 * B2 / Bmag + x3 * B3 / Bmag
# x_par_baseline = x_par_baseline[:round_size(len(x_par_baseline))+1]
# t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(x_par_baseline))+1]
# ax.plot(t_baseline[1:], 
#         x_par_baseline[1:], 
#         label = fr"Baseline", 
#         marker='o', linestyle='-', markersize=0, linewidth=0.8, color='black')
# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# tmin=43.9822971
# tmax=75.398223686155039

# ax.set_xlim([tmin,tmax])
# ax.set_ylim([20.7, 21.3])
# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$x_\parallel[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()

# # x-x0
# x10 = baseline["test_electrons"]["tracks"]["x1"][particle,0]
# x20 = baseline["test_electrons"]["tracks"]["x2"][particle,0]
# x30 = baseline["test_electrons"]["tracks"]["x3"][particle,0]
# fig, ax = plt.subplots()
# for pusher in sim.keys():
#     for dtw in sim[pusher]:
#         if pusher == "GcaCorr":
#             linestyle=(0, (12, 12))
#         else:
#             linestyle='-'
#         x1 = sim[pusher][dtw]["test_electrons"]["tracks"]["x1"][particle,:]
#         x2 = sim[pusher][dtw]["test_electrons"]["tracks"]["x2"][particle,:]
#         x3 = sim[pusher][dtw]["test_electrons"]["tracks"]["x3"][particle,:]
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,:], 
#                 np.sqrt((x1 - x10)**2 + (x2 - x20)**2 + (x3 - x30)**2), 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle=linestyle, markersize=0, linewidth=1.)# markersize=1.5

# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".0f")
# fig, ax = scale_y_ax(avrg_B, label = r"$dist[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()


# # fig, ax = plt.subplots()
# # for pusher in ["Gca"]:
# #     for dtw in ["100", "1000"]:
# #         B1  = sim[pusher][dtw]["test_electrons"]["tracks"]["B1"][particle,2:]
# #         B2  = sim[pusher][dtw]["test_electrons"]["tracks"]["B2"][particle,2:]
# #         B3  = sim[pusher][dtw]["test_electrons"]["tracks"]["B3"][particle,2:]
# #         Bmag = np.sqrt(B1**2 + B2**2 + B3**2)
# #         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
# #                 sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:] * B1/Bmag + 
# #                 sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:] * B2/Bmag +
# #                 sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:] * B3/Bmag,
# #                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$", 
# #                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
# #         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
# #         #         np.sqrt(sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:]**2 + 
# #         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:]**2 +
# #         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:]**2),
# #         #         label = fr"{pusher}, p, $\Delta t\, \Omega_e / 2\pi = {dtw.replace('_', '.')}$", 
# #         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
# #         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
# #         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p1"][particle,2:], 
# #         #         label = fr"{pusher}, $ p_1, \Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
# #         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
# #         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p2"][particle,2:], 
# #         #         label = fr"{pusher}, $ p_2, \Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
# #         # ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,2:], 
# #         #         sim[pusher][dtw]["test_electrons"]["tracks"]["p3"][particle,2:], 
# #         #         label = fr"{pusher}, $ p_3, \Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #         #         marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
        
# # # --- B on its own axis (right), same color as its line ---
# # ax2 = ax.twinx()

# # tB  = sim["Gca"]["100"]["test_electrons"]["tracks"]["t"][particle,2:]
# # B1  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B1"][particle,2:]
# # B2  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B2"][particle,2:]
# # B3  = sim["Gca"]["100"]["test_electrons"]["tracks"]["B3"][particle,2:]
# # Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

# # lnB, = ax2.plot(tB, Bmag, label=r"$B$", marker='o', linestyle='-',
# #                 markersize=1.5, linewidth=0.8, color='purple')

# # cB = lnB.get_color()
# # ax2.spines["right"].set_color(cB)
# # ax2.tick_params(axis='y', colors=cB)
# # ax2.yaxis.label.set_color(cB)
# # ax2.set_ylabel(r"$B$")

# # # ax.set_xlim([0, 40])

# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))
# # ax2.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# # ax.set_ylabel(r"$p_\parallel[c]$")

# # # legend including B
# # h1, l1 = ax.get_legend_handles_labels()
# # h2, l2 = ax2.get_legend_handles_labels()
# # ax.legend(h1 + h2, l1 + l2, loc='center left', bbox_to_anchor=(1, 0.5))
# # ax.grid()

# # plt.show()



# # # P parallel
# # fig, ax = plt.subplots()
# # for pusher in sim.keys():
# #     for dtw in sim[pusher]:
# #         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
# #                 p_parallel(sim, pusher, dtw, particle)[1:], 
# #                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
# #                 marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

# # # ax.set_xlim([0, 40])

# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# # fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# # ax.set_ylabel(r"$p_\parallel[c]$")

# # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# # plt.show()



# # p_parallel / rel error
# fig, ax = plt.subplots()
# index_min = 1

# B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
# B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
# B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
# Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

# p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
# p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
# p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

# p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
# p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
# t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(p_par_baseline))+1]
# for pusher in sim.keys():
#     x = []
#     y = []
#     err = []
#     for dtw in sim[pusher]:
#         step = int(float(dtw.replace("_", ".")))
#         if step < 1:
#             step = 1

#         p_par = p_parallel(sim, pusher, dtw, particle)
#         p_par = p_par[index_min:round_size(len(p_par))+1]
#         t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(p_par))+1]
#         x.append(float(dtw.replace("_", ".")))
#         em, se = mean_rel_err_with_ar1(abs((p_par - p_par_baseline[step::step])/p_par_baseline[step::step]))
#         y.append(em)
#         #error with autocorrelation
#         err.append(se)
#     if pusher == "GcaCorr":
#         linestyle=(0, (4, 4))
#     else:
#         linestyle='-'
#     ax.errorbar(
#         x, y, yerr=err,
#         label=fr"{pusher}",
#         fmt='o',
#         linestyle=linestyle,
#         markersize=3.5,
#         linewidth=1.2,
#         capsize=3,
#         elinewidth=1.0,
#     )

# ax.set_yscale('log')
# ax.set_xscale('log') 
# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax.set_ylabel(r"$p_\parallel$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()







# # P parallel
# fig, ax = plt.subplots()
# # index_min = 1

# B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
# B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
# B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
# Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

# p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
# p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
# p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

# p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
# # p_par_baseline = p_par_baseline[:round_size(len(p_par_baseline))+1]
# for pusher in ["Gca"]:
#     for dtw in sim[pusher]:
#         ax.plot(sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,1:], 
#                 p_parallel(sim, pusher, dtw, particle)[1:], 
#                 label = fr"{pusher}, $\Delta t\, \Omega_e / 2\pi = {dtw.replace("_", ".")}$", 
#                 marker='o', linestyle='-', markersize=0, linewidth=0.8)

# ax.plot(baseline["test_electrons"]["tracks"]["t"][particle,1:], 
#         p_par_baseline[1:], 
#         label = fr"Baseline", 
#         marker='o', linestyle='-', markersize=0, linewidth=0.8, color='black')
# # ax.set_xlim([0, 40])

# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax, fmt=".1f")
# ax.set_ylabel(r"$p_\parallel[c]$")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()



# # F_parallel~ / rel error
# fig, ax = plt.subplots()
# index_min = 1

# B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
# B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
# B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
# Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

# p1 = baseline["test_electrons"]["tracks"]["p1"][particle,:]
# p2 = baseline["test_electrons"]["tracks"]["p2"][particle,:]
# p3 = baseline["test_electrons"]["tracks"]["p3"][particle,:]

# p_par_baseline = p1 * B1 / Bmag + p2 * B2 / Bmag + p3 * B3 / Bmag
# t = baseline["test_electrons"]["tracks"]["t"][particle,:]

# slope_baseline, intercept_baseline, _, _, err_baseline = linregress(t[1:], p_par_baseline[1:])
# for pusher in sim.keys():
#     x = []
#     y = []
#     err = []
#     for dtw in sim[pusher]:
#         index = sim[pusher][dtw]["test_electrons"]["tracks"]["t"].shape[1]
#         t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle]
#         slope, intercept, _, _, stderr = linregress(t[1:index], p_parallel(sim, pusher, dtw, particle)[1:index])
#         x.append(float(dtw.replace("_", ".")))
#         y.append(abs((slope - slope_baseline) / slope_baseline))
#         err.append(np.sqrt((stderr /slope_baseline)**2 + (slope * err_baseline / slope_baseline**2)**2))

#     if pusher == "GcaCorr":
#         linestyle=(0, (4, 4))
#         # linestyle='-'
#     else:
#         linestyle='-'
#     ax.errorbar(
#         x, y, yerr=err,
#         label=fr"{pusher}",
#         fmt='o',
#         linestyle=linestyle,
#         markersize=3.5,
#         linewidth=1.2,
#         capsize=3,
#         elinewidth=1.0,
#     )

# ax.set_yscale('log')
# ax.set_xscale('log') 
# # ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# # ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
# ax.set_ylabel(r"$F_\parallel$ relative error")

# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()


# x_parallel / rel error
fig, ax = plt.subplots()
index_min = 1

B1  = baseline["test_electrons"]["tracks"]["B1"][particle,:]
B2  = baseline["test_electrons"]["tracks"]["B2"][particle,:]
B3  = baseline["test_electrons"]["tracks"]["B3"][particle,:]
Bmag = np.sqrt(B1**2 + B2**2 + B3**2)

x1 = baseline["test_electrons"]["tracks"]["x1"][particle,:]
x2 = baseline["test_electrons"]["tracks"]["x2"][particle,:]
x3 = baseline["test_electrons"]["tracks"]["x3"][particle,:]

x_par_baseline = x1 * B1 / Bmag + x2 * B2 / Bmag + x3 * B3 / Bmag
x_par_baseline = x_par_baseline[:round_size(len(x_par_baseline))+1]
t_baseline = baseline["test_electrons"]["tracks"]["t"][particle,:round_size(len(x_par_baseline))+1]
for pusher in sim.keys():
    x = []
    y = []
    err = []
    for dtw in sim[pusher]:
        step = int(float(dtw.replace("_", ".")))
        if step < 1:
            step = 1

        x_par = x_parallel(sim, pusher, dtw, particle)
        x_par = x_par[index_min:round_size(len(x_par))+1]
        t = sim[pusher][dtw]["test_electrons"]["tracks"]["t"][particle,index_min:round_size(len(x_par))+1]
        x.append(float(dtw.replace("_", ".")))
        em, se = mean_rel_err_with_ar1(abs((x_par - x_par_baseline[step::step])/x_par_baseline[step::step]))
        y.append(em)
        #error with autocorrelation
        err.append(se)
    if pusher == "GcaCorr":
        linestyle=(0, (4, 4))
    else:
        linestyle='-'
    ax.errorbar(
        x, y, yerr=err,
        label=fr"{pusher}",
        fmt='o',
        linestyle=linestyle,
        markersize=3.5,
        linewidth=1.2,
        capsize=3,
        elinewidth=1.0,
    )

ax.vlines(dtw_mirror * avrg_B / 2.0 / np.pi, ymin=1e-6, ymax=1e-2, colors='gray', linestyles='dashed', label=r"$L_M / v_i^{GC}$")
ax.set_yscale('log')
ax.set_xscale('log') 
# ax.xaxis.set_major_locator(plt.MaxNLocator(4))
# ax.yaxis.set_major_locator(plt.MaxNLocator(4))

ax.set_xlabel(r"$\Delta t \, \Omega_e / 2\pi$")
ax.set_ylabel(r"$x_\parallel$ relative error")

ax.legend()
ax.grid()
plt.tight_layout()
plt.show()



fig, ax = None, None
tmin=43.9822971
tmax=75.398223686155039
dtw = "100"

fig, ax = plot_trajectory(sim["Boris"][dtw], fr"Boris", particle=particle, avrg_B=avrg_B, tmin=tmin, tmax=tmax, fig=fig, ax=ax)
fig, ax = plot_trajectory(sim["Gca"][dtw], fr"Gca", particle=particle, avrg_B=avrg_B, tmin=tmin, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=4.5)
fig, ax = plot_trajectory(sim["GcaCorr"][dtw], fr"GcaCorr", particle=particle, avrg_B=avrg_B, tmin=tmin, tmax=tmax, fig=fig, ax=ax, markersize=0, linewidth=4.5, linestyle=(0, (2.8, 2.8)))
fig, ax = plot_trajectory(baseline, "Baseline", particle=particle, avrg_B=avrg_B, color="black", markersize=0, linewidth=1., tmin=tmin, tmax=tmax, fig=fig, ax=ax)


ax.view_init(elev=15, azim=-40)
# ax.view_init(elev=0, azim=270)
# ax.tick_params(axis='x', pad=15)
ax.tick_params(axis='y', pad=5)
ax.tick_params(axis='z', pad=15)

ax.legend(loc='upper left')
ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))
ax.zaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax, fmt=".0f")

ax.zaxis.set_rotate_label(False)
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 20
ax.zaxis.labelpad = 1
ax.set_zlabel("")
# add a manual label in axes (2D) coords: (x,y) in [0,1]
ax.text2D(1.07, 0.56, r"$x_3[c/\Omega_e]$", transform=ax.transAxes,
          rotation=0, va="center", ha="center")
plt.show()

In [ ]:
sim["Gca"]["1000"]["test_electrons"]["tracks"]["t"][particle]

In [ ]:



u = np.sqrt(0.7**2 + 0.7**2 + 0.7**2)
print("u = ", u, "\t\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))
u = np.sqrt(0.7**2 + 0.7**2 + 0.1**2)
print("u = ", u, "\t\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.5**2 + 0.5**2 + 0.1**2)
print("u = ", u, "\t\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.1**2 + 0.1**2 + 0.05**2)
print("u = ", u, "\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.05**2 + 0.05**2 + 0.001**2)
print("u = ", u, "\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

u = np.sqrt(0.05**2 + 0.7**2 + 0.7**2)
print("u = ", u, "\tv = ", UtoV(u), "\tGamma = ", UtoGamma(u))

np.sqrt(0.7**2 + 0.7**2)

In [ ]:
sim[pusher][dtw]["test_electrons"]["tracks"].units